# Pickle

Welcome to `pickle`, the most unsecure data serialization format.

Let's define a simple data structure.

In [1]:
class Cons:
    """
    A pair of values. `car` is the first value, `cdr` is the second
    value.
    """
    def __init__(self, car, cdr):
        self.car = car
        self.cdr = cdr

    def __str__(self):
        return f"({self.car} . {self.cdr})"

list = Cons(1, Cons(2, Cons(3, None)))

print(f"list = {list}")

list = (1 . (2 . (3 . None)))


Now we import `loads()` and `dumps()` from the `pickle` module. Let's
try using `dumps()` on the object we created earlier.

In [2]:
from pickle import loads, dumps

data = dumps(list)

print(f"data = {data}")
print(f"type(data) = {type(data)}")
print(f"len(data) = {len(data)} bytes")

data = b'\x80\x04\x95L\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x04Cons\x94\x93\x94)\x81\x94}\x94(\x8c\x03car\x94K\x01\x8c\x03cdr\x94h\x02)\x81\x94}\x94(h\x05K\x02h\x06h\x02)\x81\x94}\x94(h\x05K\x03h\x06Nububub.'
type(data) = <class 'bytes'>
len(data) = 87 bytes


We can now deserialize `data` by passing it to `loads()`.

In [3]:
loaded_list = loads(data)

print(loaded_list)

(1 . (2 . (3 . None)))


## Cooler stuff

`pickle` can do much more than simply packing objects into bytes
and back. In this section we will explore its possibilities.

For now of particular interest is the `pickletools` sibling module.

In [4]:
import pickletools

pickletools.dis(data)

    0: \x80 PROTO      4
    2: \x95 FRAME      76
   11: \x8c SHORT_BINUNICODE '__main__'
   21: \x94 MEMOIZE    (as 0)
   22: \x8c SHORT_BINUNICODE 'Cons'
   28: \x94 MEMOIZE    (as 1)
   29: \x93 STACK_GLOBAL
   30: \x94 MEMOIZE    (as 2)
   31: )    EMPTY_TUPLE
   32: \x81 NEWOBJ
   33: \x94 MEMOIZE    (as 3)
   34: }    EMPTY_DICT
   35: \x94 MEMOIZE    (as 4)
   36: (    MARK
   37: \x8c     SHORT_BINUNICODE 'car'
   42: \x94     MEMOIZE    (as 5)
   43: K        BININT1    1
   45: \x8c     SHORT_BINUNICODE 'cdr'
   50: \x94     MEMOIZE    (as 6)
   51: h        BINGET     2
   53: )        EMPTY_TUPLE
   54: \x81     NEWOBJ
   55: \x94     MEMOIZE    (as 7)
   56: }        EMPTY_DICT
   57: \x94     MEMOIZE    (as 8)
   58: (        MARK
   59: h            BINGET     5
   61: K            BININT1    2
   63: h            BINGET     6
   65: h            BINGET     2
   67: )            EMPTY_TUPLE
   68: \x81         NEWOBJ
   69: \x94         MEMOIZE    (as 9)
   70: }       

`pickletools.dis()` prints the internal structure of a pickle object.
This allows us to get an idea of how our data is stored on a very low
level.

We can even programmatically iterate over a series of opcodes using
`pickletools.genops()`. Let's utilize this feature to see all opcodes
used in our data and their description.

In [5]:
opcodes = {}

for (opcode, arg, pos) in pickletools.genops(data):
    opcodes[opcode.name] = opcode.doc

opcodes

{'PROTO': 'Protocol version indicator.\n\n      For protocol 2 and above, a pickle must start with this opcode.\n      The argument is the protocol version, an int in range(2, 256).\n      ',
 'FRAME': 'Indicate the beginning of a new frame.\n\n      The unpickler may use this opcode to safely prefetch data from its\n      underlying stream.\n      ',
 'SHORT_BINUNICODE': 'Push a Python Unicode string object.\n\n      There are two arguments:  the first is a 1-byte little-endian signed int\n      giving the number of bytes in the string.  The second is that many\n      bytes, and is the UTF-8 encoding of the Unicode string.\n      ',
 'MEMOIZE': 'Store the stack top into the memo.  The stack is not popped.\n\n      The index of the memo location to write is the number of\n      elements currently present in the memo.\n      ',
 'STACK_GLOBAL': 'Push a global object (module.attr) on the stack.\n      ',
 'EMPTY_TUPLE': 'Push an empty tuple.',
 'NEWOBJ': 'Build an object instance.\n\n   

So pickle isn't just convenient, it's wondrously transparent,
interactive, and self-documenting.

The only downside of piclke is that every other serialization system
from now on will seem bland compared to pickle.